<a href="https://colab.research.google.com/github/linhhvo/emotion-classifier/blob/main/VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, Nadam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory 
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16, VGG19, vgg16
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import sys
import os
import random

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!unzip drive/MyDrive/images.zip > /dev/null

In [ ]:
labels = os.listdir('train')
labels

['neutral', 'angry', 'sad', 'fear', 'happy', 'disgust', 'surprise']

In [ ]:
np.random.seed(212)
tf.random.set_seed(101)

In [ ]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '') # for validation results
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input) \
    .flow_from_directory(directory='train', target_size=(224,224), classes=labels)
validate_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input) \
    .flow_from_directory(directory='validation', target_size=(224,224), classes=labels, shuffle=False)

test_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input) \
    .flow_from_directory(directory='drive/MyDrive/test', target_size=(224,224), classes=labels)

Found 20725 images belonging to 7 classes.
Found 2000 images belonging to 7 classes.
Found 193 images belonging to 7 classes.


In [ ]:
# vgg16 = VGG16(pooling='max', include_top=False)
vgg16 = VGG16()

for layer in vgg16.layers:
  layer.trainable = False

vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model = Sequential()

for layer in vgg16.layers[:-1]:
  model.add(layer)

# for layer in vgg16.layers:
#   model.add(layer)

# model.add(Flatten())

# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.5))

# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.5))

# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.5))

model.add(Dense(7, activation='softmax', name='predictions'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [ ]:
model.compile(
    # optimizer=Nadam(learning_rate=0.0001), 
    optimizer='nadam',
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

In [ ]:
earlystopper = EarlyStopping(
    patience=20,
    restore_best_weights=True,
    verbose=1)

history = model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=validate_batches,
    validation_steps=len(validate_batches),
    epochs=40
    # callbacks=[earlystopper]
)

Epoch 1/40
648/648 [==============================] - 93s 142ms/step - loss: 1.7425 - accuracy: 0.3438 - val_loss: 1.8404 - val_accuracy: 0.3490
Epoch 2/40
648/648 [==============================] - 94s 145ms/step - loss: 1.4584 - accuracy: 0.4570 - val_loss: 2.0250 - val_accuracy: 0.3755
Epoch 3/40
648/648 [==============================] - 94s 146ms/step - loss: 1.3421 - accuracy: 0.4957 - val_loss: 1.7361 - val_accuracy: 0.3795
Epoch 4/40
648/648 [==============================] - 94s 146ms/step - loss: 1.2571 - accuracy: 0.5249 - val_loss: 1.8074 - val_accuracy: 0.3820
Epoch 5/40
648/648 [==============================] - 94s 146ms/step - loss: 1.2087 - accuracy: 0.5439 - val_loss: 1.9396 - val_accuracy: 0.3770
Epoch 6/40
648/648 [==============================] - 94s 146ms/step - loss: 1.1611 - accuracy: 0.5654 - val_loss: 1.7263 - val_accuracy: 0.4030
Epoch 7/40
333/648 [==============>...............] - ETA: 41s - loss: 1.1314 - accuracy: 0.5718

In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plot_graphs(history, 'accuracy')
plt.subplot(1,2,2)
plot_graphs(history, 'loss')

In [ ]:
train_loss, train_acc = model.evaluate(train_batches, verbose=0)
print('Train Accuracy: {:.2f}'.format(train_acc))

valid_loss, valid_acc = model.evaluate(validate_batches, verbose=0)
print('Validation Accuracy: {:.2f}'.format(valid_acc))

test_loss, test_acc = model.evaluate(test_batches, verbose=0)
print('Test Accuracy: {:.2f}'.format(test_acc))